In [1]:
import hexathello.aiPlayers as aiPlayers
import hexathello.AutoPlayer as autoPlayer
import hexathello.Engine as engine
import hexathello.history as history
import hexathello.jable as jable
import hexathello.printing as printing

import numpy as np

from os import path
import os
import math
import tensorflow as tf

from os import listdir
from os.path import join, isfile


# -- Settings
game_size: int = 5
player_count: int = 2

In [3]:
matchup_dir = "data/history/matchups_final"

# update with new JSON file
matchup_files = [f for f in listdir(matchup_dir) if f.endswith("final.json")]
print(matchup_files)

agent_ids = [
    "1x720_total720_size5_players2_final",
    "2x360_total720_size5_players2_final",
    "3x240_total720_size5_players2_final",
    "4x180_total720_size5_players2_final",
    "5x144_total720_size5_players2_final",
    "6x120_total720_size5_players2_final",
    "7x102_total714_size5_players2_final"
]

['all_data_final.json']


In [4]:
for ai_id in agent_ids:
    print(f"\n Retraining agent: {ai_id}")

    training_pov = []

    relevant_files = [f for f in matchup_files if ai_id in f]

    match_frames = []
    for fname in matchup_files:
        full_path = join(matchup_dir, fname)
        
        history_fromDisk: jable.JyFrame = jable.read_file(
            full_path
        )
    
        history_decoded: jable.JyFrame = history.history_fromInt(
            history_fromDisk
        )
    
        if len(history_decoded) < 20:
            print('skipping')
            continue
    
    
        povHistory: jable.JyFrame = history.povHistory_from_literalHistory(
            history_decoded
        )

        del history_decoded
        del history_fromDisk

        match_frames.append(povHistory)


        input_size: int = len(povHistory[0, 'board_state'])
        output_size: int = len(povHistory[0, 'player_action'])

    keras_path = join("data", "ai", "vary_dimensions", f"{ai_id}.keras")
    if not path.isfile(keras_path):
        print(f"Model not found at {keras_path}")
        continue

    brain_model = tf.keras.models.load_model(keras_path)
    os.makedirs(os.path.dirname(keras_path), exist_ok=True)

    ai_agent = aiPlayers.KerasHexAgent(
        size=game_size,
        player_count=player_count,
        brain=brain_model,
        player_id=None,
        ai_id=ai_id
    )

    checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
        filepath=keras_path,
        monitor='loss',
        mode='min',
        save_best_only=True
    )

    training_pov_frame: jable.JyFrame = jable.JyFrame(training_pov)

    for povHistory in match_frames:
        ai_agent.train(
            game_history=povHistory,
            epochs=10,
            callbacks=[checkpoint_cb]
        )

    print(f"Retrained and saved model as {keras_path}")



 Retraining agent: 1x720_total720_size5_players2_final
Epoch 1/10
3906/3906 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: 1.3628
Epoch 2/10
3906/3906 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: 1.0300
Epoch 3/10
3906/3906 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: 0.9570
Epoch 4/10
3906/3906 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: 0.9212
Epoch 5/10
3906/3906 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: 0.8978
Epoch 6/10
3906/3906 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: 0.8793
Epoch 7/10
3906/3906 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: 0.8679
Epoch 8/10
3906/3906 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: 0.8500
Epoch 9/10
3906/3906 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: 0.8398
Epoch 10/10
3906/3906 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: 0.8399
Retrained and saved model as data\ai\vary_dimensions\1x720_total720_size5_players2_final.keras

 Retraining agent: 2x360_total720_size5_players2_final
Epoch 1/10
3906/3906 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - loss: 1.3561
Epoch 2/10
3906/3906 ━━━━━━━━━━━